In [9]:
import pandas as pd
from lib.data_utils import fetch_and_prepare_data

sample_size = 10_000
try:
    data_df = pd.read_csv('data_samples/paragraph-questions-10k.csv')
    print('Dataset already exists.')
except FileNotFoundError:
    data_df = await fetch_and_prepare_data(sample_size)
    data_df.to_csv('data_samples/paragraph-questions-10k.csv', index=False)
data_df

,_sa_instance_state_question,paragraph_id,scope,text_question,timestamp,downvote,id_question,context,author_id,upvote,...,setting,id_answer,text,_sa_instance_state_rating,text_rating,value,author_id_rating,id_rating,answer_id,timestamp_rating
0,<sqlalchemy.orm.state.InstanceState object at ...,646164,single-paragraph,What nickname was given to Isaac Brock because...,2024-05-11 18:18:43,0,702046,"In an article about 'Isaac Brock', section 'Ov...",8,0,...,ic,1435936,The nickname given to Isaac Brock because of h...,<sqlalchemy.orm.state.InstanceState object at ...,This answer is fully accurate and detailed. It...,5,7,1435749,1435936,2024-05-11 18:19:18
1,<sqlalchemy.orm.state.InstanceState object at ...,767883,single-paragraph,What family of dogs does the Welsh Springer Sp...,2024-05-09 07:18:44,0,575465,"In an article about 'Welsh Springer Spaniel', ...",8,0,...,ic,1165490,The Welsh Springer Spaniel belongs to the span...,<sqlalchemy.orm.state.InstanceState object at ...,The answer provided is fully accurate and deta...,5,7,1165306,1165490,2024-05-09 07:19:00
2,<sqlalchemy.orm.state.InstanceState object at ...,160806,single-paragraph,What is true about the role that Cid plays in ...,2024-05-14 10:35:33,0,805377,"In an article about 'Final Fantasy', section '...",8,0,...,ic,1662590,Cid's role in different Final Fantasy games va...,<sqlalchemy.orm.state.InstanceState object at ...,The answer is mostly correct as it mentions th...,4,7,1662399,1662590,2024-05-14 10:36:28
3,<sqlalchemy.orm.state.InstanceState object at ...,341441,single-paragraph,"Who co-created ""The Unadulterated Cat"", a humo...",2024-05-21 06:25:20,0,1000970,"In an article about 'Terry Pratchett', section...",8,0,...,zs,2118796,"Terry Pratchett and Gray Jolliffe co-created ""...",<sqlalchemy.orm.state.InstanceState object at ...,This answer is fully accurate and detailed. It...,5,7,2118596,2118796,2024-05-21 06:25:44
4,<sqlalchemy.orm.state.InstanceState object at ...,203290,single-paragraph,"Why are samples older than 50,000 years diffic...",2024-05-20 16:12:46,0,989548,"In an article about 'Radiocarbon dating', sect...",8,0,...,ic,2089993,"Samples older than 50,000 years are difficult ...",<sqlalchemy.orm.state.InstanceState object at ...,The answer provided directly quotes the releva...,5,7,2089795,2089993,2024-05-20 16:13:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,<sqlalchemy.orm.state.InstanceState object at ...,718131,single-paragraph,What distinction does the Stonewall Inn hold a...,2024-05-10 17:12:53,0,650315,"In an article about 'Stonewall riots', section...",8,0,...,ic,1324903,The Stonewall Inn holds the distinction of bei...,<sqlalchemy.orm.state.InstanceState object at ...,The answer accurately states that the Stonewal...,5,7,1324723,1324903,2024-05-10 17:13:49
9996,<sqlalchemy.orm.state.InstanceState object at ...,765757,single-paragraph,"On what date was the Doctor Who episode ""Closi...",2024-05-04 09:42:51,0,267341,In an article about 'Closing Time ( Doctor Who...,8,0,...,ic,533647,"The Doctor Who episode ""Closing Time"" was firs...",<sqlalchemy.orm.state.InstanceState object at ...,This answer is fully accurate and detailed. It...,5,7,533485,533647,2024-05-04 09:43:10
9997,<sqlalchemy.orm.state.InstanceState object at ...,626545,single-paragraph,"What song by Jo Stafford, Frank Sinatra, and T...",2024-05-16 15:29:41,0,883788,"In an article about 'Jo Stafford', section 'Th...",8,0,...,zs,1840626,"The answer is ""I'll Never Smile Again"".",<sqlalchemy.orm.state.InstanceState object at ...,The answer is fully accurate and detailed. It ...,5,7,1840425,1840626,2024-05-16 15:29:48
9998,<sqlalchemy.orm.state.InstanceState object at ...,322766,single-paragraph,What are the names of the two athletics clubs ...,2024-05-07 22:31:07,0,497457,"In an article about 'Kingston upon Hull', sect...",8,0,...,zs,1000314,The two athletics clubs that share a base at t...,<sqlalchemy.orm.state.InstanceState object 

In [10]:
data_df.iloc[0]

_sa_instance_state_question     <sqlalchemy.orm.state.InstanceState object at ...
paragraph_id                                                               646164
scope                                                            single-paragraph
text_question                   What nickname was given to Isaac Brock because...
timestamp                                                     2024-05-11 18:18:43
downvote                                                                        0
id_question                                                                702046
context                         In an article about 'Isaac Brock', section 'Ov...
author_id                                                                       8
upvote                                                                          0
turns                                                                      single
_sa_instance_state_paragraph    <sqlalchemy.orm.state.InstanceState object at ...
page_name       

In [11]:
# Average context length per question
data_df['context_length'] = data_df['context'].apply(lambda x: len(x.split())) + data_df['text_paragraph'].apply(lambda x: len(x.split()))
average_context_length = data_df['context_length'].mean()
print(f"Average context length per question: {average_context_length}")

# std deviation of context length
std_dev_context_length = data_df['context_length'].std()
print(f"Standard deviation of context length: {std_dev_context_length}")

# Total context length for retrieval systems
total_context_length = data_df['context_length'].sum()
print(f"Total context length: {total_context_length}")

# Number of unique contexts
num_unique_contexts = data_df['context'].nunique()
print(f"Number of unique contexts: {num_unique_contexts}")

Average context length per question: 125.5295
Standard deviation of context length: 74.28393056364209
Total context length: 1255295
Number of unique contexts: 9529


In [12]:
data_df['question_length'] = data_df['text_question'].apply(lambda x: len(x.split()))
average_question_length = data_df['question_length'].mean()
std_question_length = data_df['question_length'].std()
print(f"Average question length: {average_question_length}")
print(f"Standard deviation of question length: {std_question_length}")

Average question length: 16.9011
Standard deviation of question length: 4.961691307211968


In [13]:
# Answer length stats
data_df['answer_length'] = data_df['text'].apply(lambda x: len(x.split()))
average_answer_length = data_df['answer_length'].mean()
std_answer_length = data_df['answer_length'].std()
print(f"Average answer length: {average_answer_length}")
print(f"Standard deviation of answer length: {std_answer_length}")

Average answer length: 23.1824
Standard deviation of answer length: 22.00491653870079


In [7]:
from nltk.corpus import wordnet as wn

# Function to get synsets
def get_synsets(text):
    words = text.split()
    synsets = [wn.synsets(word) for word in words]
    return synsets

data_df['question_synsets'] = data_df['text_question'].apply(get_synsets)

# Example analysis of synsets
sample_synsets = data_df['question_synsets'].head()
print(sample_synsets)

0    [[], [Synset('make.v.01'), Synset('perform.v.0...
1    [[], [Synset('literary.a.01'), Synset('literar...
2    [[], [Synset('bash.n.02'), Synset('do.n.02'), ...
3    [[], [Synset('distinctive.s.01'), Synset('clas...
4    [[], [Synset('operation.n.01'), Synset('operat...
Name: question_synsets, dtype: object


In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc]
    return lemmatized

data_df['lemmatized_question'] = data_df['text_question'].apply(lemmatize_text)

# Dictionary coverage counting
all_words = [word for lemmatized in data_df['lemmatized_question'] for word in lemmatized]
unique_words = set(all_words)
word_count = len(unique_words)
print(f"Total unique words: {word_count}")

Total unique words: 21668


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(data_df['text_question'])

# LDA for topic modeling
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(tfidf_matrix)

# Display topics
for index, topic in enumerate(lda.components_):
    print(f"Topic #{index}:")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Topic #0:
['do', 'is', 'how', 'and', 'in', 'to', 'type', 'what', 'the', 'of']
Topic #1:
['new', 'on', 'was', 'and', 'in', 'did', 'what', 'of', 'to', 'the']
Topic #2:
['by', 'title', 'was', 'album', 'released', 'in', 'what', 'of', 'is', 'the']
Topic #3:
['where', 'which', 'and', 'are', 'is', 'to', 'in', 'what', 'of', 'the']
Topic #4:
['date', 'which', 'year', 'did', 'was', 'on', 'of', 'what', 'in', 'the']
Topic #5:
['wrote', 'to', 'what', 'about', 'how', 'did', 'who', 'of', 'in', 'the']
Topic #6:
['was', 'during', 'and', 'who', 'what', 'to', 'in', 'did', 'of', 'the']
Topic #7:
['role', 'film', 'was', 'and', 'which', 'what', 'who', 'of', 'in', 'the']
Topic #8:
['did', 'was', 'according', 'and', 'in', 'is', 'what', 'of', 'to', 'the']
Topic #9:
['and', 'to', 'what', 'which', 'did', 'for', 'of', 'was', 'in', 'the']


In [12]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

# 4-bag-of-words
vectorizer_4 = CountVectorizer(ngram_range=(4, 4))
bow_4 = vectorizer_4.fit_transform(data_df['text_question'])
bow_4_freq = dict(zip(vectorizer_4.get_feature_names_out(), bow_4.sum(axis=0).tolist()[0]))

# 8-bag-of-words
vectorizer_8 = CountVectorizer(ngram_range=(8, 8))
bow_8 = vectorizer_8.fit_transform(data_df['text_question'])
bow_8_freq = dict(zip(vectorizer_8.get_feature_names_out(), bow_8.sum(axis=0).tolist()[0]))

# Find highest frequency top 8
top_8_bow_4 = Counter(bow_4_freq).most_common(32)
top_8_bow_8 = Counter(bow_8_freq).most_common(32)
print("Top 8 4-bag-of-words:", top_8_bow_4)
print("Top 8 8-bag-of-words:", top_8_bow_8)

Top 8 4-bag-of-words: [('in what year did', 197), ('the name of the', 142), ('in what year was', 141), ('the title of the', 133), ('what is the name', 120), ('is the name of', 114), ('what is the title', 112), ('is the title of', 111), ('on what date did', 84), ('what year was the', 73), ('on what date was', 70), ('what year did the', 63), ('in the united states', 59), ('what was the title', 58), ('was the title of', 56), ('what is the primary', 51), ('in relation to the', 49), ('what was the original', 46), ('what was the name', 44), ('in the context of', 43), ('was the name of', 41), ('what date did the', 40), ('and what was the', 39), ('played the role of', 38), ('what is the typical', 37), ('what was the outcome', 37), ('what date was the', 34), ('what was the primary', 33), ('was the outcome of', 31), ('during the battle of', 30), ('in terms of its', 28), ('what happens to the', 27)]
Top 8 8-bag-of-words: [('what is the term used to describe the', 10), ('is the name of the charact

In [13]:
for i, (word, freq) in enumerate(top_8_bow_4):
    print(f"4-bag-of-words #{i}: {word} ({freq})")

4-bag-of-words #0: in what year did (197)
4-bag-of-words #1: the name of the (142)
4-bag-of-words #2: in what year was (141)
4-bag-of-words #3: the title of the (133)
4-bag-of-words #4: what is the name (120)
4-bag-of-words #5: is the name of (114)
4-bag-of-words #6: what is the title (112)
4-bag-of-words #7: is the title of (111)
4-bag-of-words #8: on what date did (84)
4-bag-of-words #9: what year was the (73)
4-bag-of-words #10: on what date was (70)
4-bag-of-words #11: what year did the (63)
4-bag-of-words #12: in the united states (59)
4-bag-of-words #13: what was the title (58)
4-bag-of-words #14: was the title of (56)
4-bag-of-words #15: what is the primary (51)
4-bag-of-words #16: in relation to the (49)
4-bag-of-words #17: what was the original (46)
4-bag-of-words #18: what was the name (44)
4-bag-of-words #19: in the context of (43)
4-bag-of-words #20: was the name of (41)
4-bag-of-words #21: what date did the (40)
4-bag-of-words #22: and what was the (39)
4-bag-of-words #23: